In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate, Lambda
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras import backend as K

Using TensorFlow backend.


In [2]:
n_samples = 20000
sample_inputs = []
sample_outputs = []
n_tokens = 61

patterns = [
    lambda i,l: list(range(i, i+l)),
    lambda i,l: list(range(i, i-l, -1))
]

for n in range(n_samples):
    sample_in = []
    sample_out = []
    for line_number,length in enumerate([5,7,5]):
        i = np.random.randint(10,50)
        #line = list(range(i, i+3))
        pattern = np.random.randint(len(patterns))
        line = patterns[pattern](i,length)
        '''if line_number == 0:
            sample_in.append([line[0]] + line)
        else:
            sample_in.append([0] + line)'''
        
        sample_in.append([line[0]] + line + [60]*(9-length))
            
        sample_out.append(line + [60]*(10-length))
        
        
        #sample.append(np.arange(i, i+3))
    #samples.append(sample)
    sample_inputs.append(sample_in)
    sample_outputs.append(sample_out)
    
X = np_utils.to_categorical(sample_inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(sample_outputs, num_classes=n_tokens)

In [3]:
latent_dim = 128

# Keras Model

inputs = [Input(shape=(None, n_tokens)) for i in range(3)]
lstms = [LSTM(latent_dim, return_state=True, return_sequences=True) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[lstm_h[i-1], lstm_c[i-1]])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i])
        
denses = [Dense(n_tokens, activation='softmax') for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

# Enforce number of chars per line
counter = Lambda(lambda x: K.sum(K.cast(K.less(K.argmax(x, axis=2), 60), K.floatx()), axis=1, keepdims=True))
counts = [counter(i) for i in outputs]

model = Model(inputs, outputs + counts)

# Use categorical for the character outputs and MSE for the counts
model.compile(optimizer='rmsprop', loss=['categorical_crossentropy']*3 + ['mean_squared_error']*3)
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_1 (LSTM)                                                     [(None, None, 128), (None, 128), (None, 128 97280                   input_1[0][0]                                                 

In [4]:
model.fit(
    [X[:,0,:], X[:,1,:], X[:,2,:]], 
    [
        Y[:,0,:], Y[:,1,:], Y[:,2,:], 
        np.ones(n_samples)*5, np.ones(n_samples)*7,  np.ones(n_samples)*5
    ],
    batch_size=128, epochs=20
)

Epoch 1/20
20000/20000 [==============================] - 8s 394us/step - loss: 31.3184 - dense_1_loss: 2.0447 - dense_2_loss: 2.6206 - dense_3_loss: 1.8829 - lambda_1_loss: 3.1475
Epoch 2/20
20000/20000 [==============================] - 6s 279us/step - loss: 4.9539 - dense_1_loss: 1.3332 - dense_2_loss: 1.5602 - dense_3_loss: 1.1170 - lambda_1_loss: 0.1911
Epoch 3/20
20000/20000 [==============================] - 6s 277us/step - loss: 2.3506 - dense_1_loss: 0.8455 - dense_2_loss: 0.7210 - dense_3_loss: 0.5416 - lambda_1_loss: 0.0721
Epoch 4/20
20000/20000 [==============================] - 6s 278us/step - loss: 1.1638 - dense_1_loss: 0.5065 - dense_2_loss: 0.3010 - dense_3_loss: 0.2160 - lambda_1_loss: 0.0339
Epoch 5/20
20000/20000 [==============================] - 6s 285us/step - loss: 0.7251 - dense_1_loss: 0.3406 - dense_2_loss: 0.1487 - dense_3_loss: 0.1157 - lambda_1_loss: 0.0370
Epoch 6/20
20000/20000 [==============================] - 6s 304us/step - loss: 0.5074 - dense_1_lo

In [5]:
generator_inputs = [Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True) for i in range(3)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

# Hook up line 0
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0]
)
generator_outputs = [generator_denses[0](generator_lstm_out[0])]
generator_models = [Model(generator_inputs[0], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])]

# Set weights for line 0
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())

generator_models[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (1, None, 61)             0         
_________________________________________________________________
lstm_4 (LSTM)                [(1, None, 128), (1, 128) 97280     
_________________________________________________________________
dense_4 (Dense)              (1, None, 61)             7869      
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________


In [6]:
generator_inputs_h = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
generator_inputs_c = [None, Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]

#generator_inputs.append(Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens)))
#generator_lstms.append(LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True))


# Hook up lines 1 and 2
for i in [1, 2]:
    generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
        generator_inputs[i], initial_state=[generator_inputs_h[i], generator_inputs_c[i]]
    )
    generator_outputs.append(generator_denses[i](generator_lstm_out[i]))
#generator_denses.append(Dense(n_tokens, activation='softmax'))

    generator_models.append(Model(
        [generator_inputs[i], generator_inputs_h[i], generator_inputs_c[i]],
        [generator_outputs[i], generator_lstm_h[i], generator_lstm_c[i]]
    ))

    generator_lstms[i].set_weights(lstms[i].get_weights())
    generator_denses[i].set_weights(denses[i].get_weights())

#generator_model_line1.summary()

In [13]:
h,c = None, None
generator_models[0].reset_states()
for i in range(3):
    line = [np.random.randint(10,50)]
    
    def f():
        while True:
            if i > 0:
                yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
            else:
                yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
        
    while len(line) < 10:
        character, h, c = generator_models[i].predict_generator(f(), steps=1)
        line.append(np.argmax(character))
        
    print(line[1:line.index(60)])

[36, 37, 38, 39, 40]
[13, 14, 15, 16, 17, 18, 19]
[48, 47, 46, 45, 44]
